## Введение в анализ данных и машинное обучение на Python

### Тема 11: Анализ наборов данных «Титаник» и «Ирисы»

---


### Часть 1: «Титаник»

[Данные](http://www.machinelearning.ru/wiki/images/3/32/Titanic.zip)

[Описание данных](https://www.kaggle.com/c/titanic/data)

[Хороший пример](https://towardsdatascience.com/predicting-the-survival-of-titanic-passengers-30870ccc7e8), но на английском.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data.head()

In [ ]:
plt.hist(data['Survived'])
plt.title('Гистограмма распределения выживших пассажиров')
plt.xlabel('Выжил ли пассажир (0 -- нет, 1 -- да)')

In [ ]:
plt.scatter(data['Fare'], data['Survived'])

#### Постановка задачи регрессии


**Исследовательский вопрос:** Как цена билета зависит от: возраста индивида, класса индивида, пола индивида?

**Гипотеза:** 
1. Возраст -- положительно. 
2. Класс -- отрицательно. 
3. Пол: мужчины -- более дешёвые билеты.

**Механизм:** 
1. Чем старше индивид, тем больше его доходы и накопления, а значит, он может купить билет дороже.
2. Первый класс обладает большими удобствами, чем третий, а значит, цена билета в первый класс выше.
3. На "Титанике" было много рабочих мужчин, а женщины являлись, в основном, жёнами состоятельных людей. Поэтому мужчины более вероятно покупали более дешёвые билеты.

**Признаки:** 
1. Возраст: `Age`.
2. Класс: `Pclass`.
3. Пол: `Sex`.

**Методы:** Линейная (множественная) регрессия.

In [ ]:
# Получим необходимые признаки
data_sample = data[['Age', 'Pclass', 'Sex', 'Fare']]

In [ ]:
# Посмотрим, что получилось
data_sample

In [ ]:
# Перекодируем переменную Sex
sex = data_sample['Sex'].copy()
sex[sex == 'male'] = int(0)
sex[sex == 'female'] = int(1)
sex = pd.to_numeric(sex)
sex

In [ ]:
# Заменим столбец
data_sample = data_sample.drop('Sex', axis = 1)
data_sample['Sex'] = sex

In [ ]:
# Проверим типы столбцов
data_sample.dtypes

In [ ]:
# Проверим наличие отсутствующих значений
np.sum(data_sample.isna())

In [ ]:
# Удалим пропущенные значения
data_sample = data_sample.dropna(subset=['Age'])
# Проверим
np.sum(data_sample.isna())

In [ ]:
# Разобьём на обучающую и тестовую выборки
np.random.seed(123)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_sample[['Age', 'Pclass', 'Sex']], 
                                                    data_sample['Fare'], test_size=0.3)

In [ ]:
X_train.head()

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
print('MSE: %f\nR2: %f' % (mean_squared_error(y_test, predictions), r2_score(y_test, predictions)))

In [ ]:
model.coef_

In [ ]:
model.intercept_

#### Постановка задачи классификации


**Исследовательский вопрос:** Как факт выживания на Титанике зависит от возраста и цены билета индивида?

**Гипотеза:** 
1. Чем индивид моложе, тем более вероятно он выживет.
2. Чем цена билета индивида выше, тем более вероятно он выживет.

**Механизм:** 

1. 
    - Детей первых сажали на лодки.
    - Более молодые обычно более выносливые.
2. Кабины первых классов находились наверху, а следовательно, имели больше шансов первыми добраться до лодок.

**Признаки:** 
1. Возраст индивида (`Age`)
2. Цена билета (`Fare`)

**Методы:** 

Решающее дерево

In [ ]:
data_sample2 = data[['Survived', 'Age', 'Fare']]
np.sum(data_sample2.isna())

In [ ]:
data_sample2 = data_sample2.dropna()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier(max_depth=1)
model3 = DecisionTreeClassifier(max_depth=5)

In [ ]:
np.random.seed(123)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_sample2[['Age', 'Fare']], 
                                                    data_sample2['Survived'], test_size=0.3)

In [ ]:
model2.fit(X_train, y_train)
model3.fit(X_train, y_train)

In [ ]:
pred2 = model2.predict(X_test)
pred3 = model3.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('Глубина 1: %f \nГлубина 5: %f' % (accuracy_score(y_test, pred2), accuracy_score(y_test, pred3)))

In [ ]:
plt.scatter(X_test['Age'], X_test['Fare'], c=pred3)

In [ ]:
from sklearn import tree

axes = plt.axes((0, 0, 10, 10))
tree.plot_tree(model3, ax = axes, fontsize=14)

#### Выводы

### Часть 2: «Ирисы»

Данные: в числе наборов данных `sklearn`.

[Описание данных](https://www.kaggle.com/uciml/iris)

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
iris.keys()

In [ ]:
X = pd.DataFrame(iris['data'])
X.columns = iris['feature_names']
X.head()

In [ ]:
Y = iris['target']
Y

In [ ]:
iris['target_names']

In [ ]:
plt.scatter(X['sepal length (cm)'], X['petal width (cm)'], c=Y)

#### Постановка задачи

Классически ставится задача классификации цветков ириса на основании четырёх измерений. Задача аналитика в данном случае заключается в подборе модели, которая покажет наилучшее качество прогнозов. 

**Признаки:**

**Используемые модели:** kNN, Случайный лес

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
forest = RandomForestClassifier(n_estimators=20)

In [ ]:
np.random.seed(123)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)

In [ ]:
knn.fit(X_train, y_train)
forest.fit(X_train, y_train)

In [ ]:
pred_knn = knn.predict(X_test)
pred_forest = forest.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('kNN: %f\nRandom Forest: %f' % (accuracy_score(y_test, pred_knn), accuracy_score(y_test, pred_forest)))

#### Выводы